# s-d model + tight binding model, cluster ansatz by Yordanov

In [1]:
#Qiskit modules
import qiskit
from qiskit import QuantumRegister as Q_R
from qiskit import ClassicalRegister as C_R
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

#math modules
import math
import numpy as np
# SciPy minimizer routine
from scipy.optimize import minimize

#My libraries
import vqe_funcs

In [2]:
PI = np.pi

## Here are the functions performing Jordan-Wigner transformation for fermionc Pauli operators and kinetci energy of electrons 

Important notes:

1) This is spin-depenedent version, so each node has spin-coordinate and is described by 2 qubits.

   Spin-up state for node i is described by qubit i.

   Spin-down state for node i is described by qubit i + N, where N is the nodes number.

In [3]:
from vqe_funcs import sigma_x, sigma_y, sigma_z, kinetic_energy, full_ham

## Magnetization distribution description

In [4]:
#Uniform magnetization along z axis
nodes_number = 4
mag = []

def uniform_z(nodes_number):
    mag = []
    for i in range(nodes_number):
        mag.append([0, 0, 1])
    return mag

## Full Hamiltonian

The Hamiltonian includes kinteic energy and s-d exchage energy

In [5]:
nodes_number = 2
mag = uniform_z(nodes_number)
full_ham(nodes_number, mag, 4, 1, periodic = True)

SparsePauliOp(['IIXX', 'IIYY', 'XXII', 'YYII', 'IIXX', 'IIYY', 'XXII', 'YYII', 'IIZI', 'ZIII', 'IIIZ', 'IZII'],
              coeffs=[-0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j,
 -0.5+0.j, -2. +0.j,  2. +0.j, -2. +0.j,  2. +0.j])

In [6]:
def anzatz_qc(theta, nodes_number, electrons_number):
    qubits_number = 2 * nodes_number
    q_r = Q_R(qubits_number)
    v_qc = QuantumCircuit(q_r)
    for i in range(electrons_number):
        v_qc.x(i)
    v_qc = vqe_funcs.exc_yordanov(qubits_number, theta, v_qc)
    return v_qc

In [19]:
nodes_number = 6
electrons_number = 2
n_qubits = nodes_number * 2
len_se = n_qubits * (n_qubits - 1) / 2
len_de = n_qubits * (n_qubits - 1) * (n_qubits - 2) * (n_qubits - 3) / 4 / 2
len_tot = int(len_se + len_de)

theta = [0]
for i in range(len_tot):
    theta.append(0)

theta[8] = PI / 9

qc = anzatz_qc(theta, nodes_number, electrons_number)


In [20]:
qc.decompose().count_ops()

OrderedDict([('cx', 39162),
             ('r', 12078),
             ('u2', 8976),
             ('h', 6120),
             ('u3', 5942),
             ('u1', 198)])

In [9]:
# defining a cost function
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
def estim(theta, nodes_number, hamiltonian, electrons_number):
    v_qc = anzatz_qc(theta, nodes_number, electrons_number)
    
    estimator = StatevectorEstimator()
    job = estimator.run([(v_qc, hamiltonian)])
    estimator_expvals = job.result()[0].data.evs
    
      
    #print 
    #print('Energy: ' + str(estimator_expvals))
    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = theta
    cost_history_dict["cost_history"].append(estimator_expvals)
    return estimator_expvals

In [10]:
# Hamiltonian parameters
nodes_number = 3 #number of nodes
n_qubits = nodes_number * 2
J = -5 #s-d exchange constant
t = 2 #hopping matrix element (kinetic energy coefficient)
electrons_number = 3

#Cost history initialization
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

import random
num = random.random()
#print(num)


mag = []
mag = uniform_z(nodes_number)

#initialize the state
len_se = n_qubits * (n_qubits - 1) / 2
len_de = n_qubits * (n_qubits - 1) * (n_qubits - 2) * (n_qubits - 3) / 4 / 2
len_tot = int(len_se + len_de)
x0 = []
for i in range(len_tot):
    x0.append(random.random())

#define the Hamiltonian
hamiltonian = full_ham(nodes_number, mag, J, t, periodic = False)
#print(hamiltonian)
#Optimization
res = minimize(
        estim,
        x0,
        args=(nodes_number , hamiltonian, electrons_number),
        method="cobyla",
    )


In [11]:
q_r = Q_R(nodes_number * 2)
cl_r = C_R(nodes_number * 2)
qc_f = QuantumCircuit(q_r,cl_r)
qc_1 = anzatz_qc(res.x, nodes_number, electrons_number)
qc_f.append(qc_1, q_r)

#for i in range(bit_size):
#    qc_f.measure(i,i)
SimulatorAer = AerSimulator()

qc_f.save_statevector()

#qc_f.decompose().draw('mpl')

In [12]:
qc_f.decompose().count_ops()

OrderedDict([('cx', 881),
             ('ry', 405),
             ('h', 285),
             ('x', 183),
             ('cz', 102),
             ('rz', 45),
             ('save_statevector', 1)])

In [13]:
from qiskit.quantum_info import partial_trace, DensityMatrix
from qiskit.visualization import plot_state_city

circ = transpile(qc_f, backend = SimulatorAer)
result = SimulatorAer.run(circ,shots = 1).result()
ground_state = result.get_statevector(circ)

In [14]:
import sys
sys.path.insert(0, 'C:/Users/Oleg/Google Диск/QC/Codes/QC-qiskit-codes/Library')
import aux_func as af
n_nonzero = 0
n_states = pow(2, 2 * nodes_number)
states = []
prob = []
for i in range(n_states):
    pr = pow(abs(ground_state[i]), 2)
    if pr>1/n_states:
        n_nonzero = n_nonzero + 1
        states.append(i)
        prob.append(pr)
        print('state: ' + str(af.int_2_bin_word(i, 2 * nodes_number)) + ', prob: ' + str(pr) + ', complex amplitude: ' + str(ground_state[i]))
    
print('Energy: ' + str(res.fun))

state: 000111, prob: 0.9995382753749635, complex amplitude: (0.7069435180320157-0.706943518032014j)
Energy: -14.993953395543324
